# Random Fuzzer

In [178]:
import random
from random import randint
import string
def randomFuzzer(size=0, outputType='alphanumeric'):
    if(size == 0):
        return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(randint(0, 50)))
    if(outputType == 'string'):
        letters = string.ascii_letters
        return ''.join(random.choice(letters) for i in range(size))
    elif(outputType == 'number'):
        range_start = 10**(size-1)
        range_end = (10**size)-1
        return randint(range_start, range_end)
    elif(outputType == 'alphanumeric'):        
        return ''.join(random.choice(string.ascii_letters + string.digits) for _ in range(size))  
          
        
    

# Mutation Fuzzer

In [187]:
import random
def mutationFuzzer(s):    
    if(len(s) != 0):
        pos = random.randint(0, len(s) - 1)
        if(type(s) in ['int', 'long'] ):
            random_character = randint(0, 9)
        #elif (type(s) == 'str'):
        else:
            random_character = chr(random.randrange(32, 127))
        if bool(random.getrandbits(1)):
            return s[:pos] + random_character + s[pos:]
        else:
            if s == "":
                return s
            pos = random.randint(0, len(s) - 1)
            return s[:pos] + s[pos+1:]

# Validate Email Function

In [188]:
from email_validator import validate_email, EmailNotValidError
import sys

def ValidateEmail(email):
    try:
        v = validate_email(email)
        domain = v["domain"]       
        return str('Domain: '+ domain)
    except EmailNotValidError as e:    
        return str('Error: '+ str(e))


# Trace Method  Definition

In [181]:
coverage = set()
def traceit(frame, event, arg):
    if event == "line":
        global coverage              
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        file_name = frame.f_code.co_filename.split('\\')[-1]        
        coverage.add((file_name,function_name, lineno))
    return traceit

# Feed fuzzing inputs to Validate Email

In [189]:
def MutateFuzzValidateEmail(seed, trials, program):   
    fileError = open("fuzzMutationEmailError.txt", "w")
    filePass = open("fuzzMutationEmailPass.txt", "w")
    totalCoverage = set()
    seedDict = {}
    seedDict[seed] = 0
    
    while(len(seedDict)!=0):
        currentSeed = max(seedDict, key=seedDict.get)
        print('currentSeed: '+ currentSeed)
        for i in range(trials):
            mutatedData = mutationFuzzer(currentSeed)
            global coverage
            coverage = set()        
            sys.settrace(traceit)  # Turn on        
            result = program(mutatedData)        
            sys.settrace(None)  # Turn off
            #print(repr(coverage - totalCoverage))
            if (bool(coverage - totalCoverage)):
                seedDict[mutatedData] = len(coverage - totalCoverage)
                totalCoverage = totalCoverage.union(coverage)                                
                print('adding'+ repr(seedDict))
            if result.startswith('Error'):                        
                fileError.write((mutatedData + '=> ' + result + ' Coverage: '+ str(len(coverage)) + '\n'))
            else:                       
                filePass.write((mutatedData + ' => ' + result + ' Coverage: '+ str(len(coverage)) + '\n'))
            print(str(len(totalCoverage)))
                #seed = mutatedData
        del seedDict[currentSeed]
        print('Afterdel' + repr(seedDict))
    fileError.close()
    filePass.close()

In [190]:
MutateFuzzValidateEmail("abhi@gmail.com", 3, ValidateEmail)

currentSeed: abhi@gmail.com
adding{'abhi@gmail.com': 0, 'a1bhi@gmail.com': 549}
549
adding{'abhi@gmail.com': 0, 'a1bhi@gmail.com': 549, 'abhi@gmailcom': 3}
552
552
Afterdel{'a1bhi@gmail.com': 549, 'abhi@gmailcom': 3}
currentSeed: a1bhi@gmail.com
552
552
552
Afterdel{'abhi@gmailcom': 3}
currentSeed: abhi@gmailcom
552
552
552
Afterdel{}


In [34]:
MutateFuzzValidateEmail("jajdaadb@gmail.com", 2)

795
8


In [30]:
sys.settrace(traceit)  # Turn on        
ValidateEmail("ajdsja@gmail.com")        
sys.settrace(None)  # Turn off 
len(coverage)

981

In [9]:
runsPass

NameError: name 'runsPass' is not defined

In [93]:
def flipBitsFuzzer(input):
    bits = ''.join('{:08b}'.format(b) for b in input.encode('utf8'))
    flippedBits = ''.join('1' if x == '0' else '0' for x in bits)
    result = bytes(int(b, 2) for b in re.split('(........)', flippedBits) if b).decode('utf8')
    print(result)
    
    
    

In [220]:
mutationFuzzer("my+address@google.com")

'my+addres@google.com'

In [257]:
from email_validator import validate_email, EmailNotValidError
import sys

email = "my+address@goaklemdogle.com"

try:
    v = validate_email(email) # validate and get info
    email = v["email"] # replace with normalized form
    print(v)
except EmailNotValidError as e:
    # email is not valid, exception message is human-readable
    print('Error: '+str(e))

Error: The domain name goaklemdogle.com does not exist.


In [294]:
from email_validator import validate_email, EmailNotValidError
import sys

email = "my+address@gmail.com"
v = validate_email(email) # validate and get info
email = v["email"] # replace with normalized form

   
    

In [179]:
import os
import subprocess
result = subprocess.run(['python', 'ValidateEmail.py', 'my+address@google.com'],
                            stdin=subprocess.DEVNULL,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True)

In [200]:
#Random fuzzer on validate email
trials = 2
runs = []
file = open("fuzzRandomEmail.txt", "w")

for i in range(trials):
    data = randomFuzzer()    
    result = subprocess.run(['python', 'ValidateEmail.py', data],
                            stdin=subprocess.DEVNULL,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True)   
    
    runs.append((data + ':' + result.stdout))
file.write(repr(runs))

177

In [362]:
def runProcess():
    return subprocess.run(['python', 'ValidateEmail.py', mutatedData],
                                stdin=subprocess.DEVNULL,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                universal_newlines=True)

In [371]:
#Mutation fuzzer on validate email
def MutateFuzzValidateEmail(seed, trials):    
    runsPass = []
    runsFail = []
    fileError = open("fuzzMutationEmailError.txt", "w")
    filePass = open("fuzzMutationEmailPass.txt", "w")    

    for i in range(trials):
        mutatedData = mutationFuzzer(seed)
        coverage = set()
        sys.settrace(traceit)  # Turn on        
        #result = runProcess()
        runProcess()
        #sys.settrace(None)  # Turn off
        print(str(len(coverage)))
#         if result.stdout.startswith('Error'):
#             runsFail.append((mutatedData + ': ' + result.stdout + 'Coverage: '+ str(len(coverage))))
#             print(str(len(coverage)))
#             fileError.write((mutatedData + ': ' + result.stdout + 'Coverage: '+ str(len(coverage))))
#         else:
#             runsPass.append((mutatedData + ': ' + result.stdout + 'Coverage: '+ str(len(coverage))))
#             print(str(len(coverage)))
#             filePass.write((mutatedData + ': ' + result.stdout + 'Coverage: '+ str(len(coverage))))
        seed = mutatedData
    fileError.close()
    filePass.close()
    
    

In [400]:
#Mutation fuzzer on validate email
def MutateFuzzValidateEmail(seed, trials):    
    runsPass = []
    runsFail = []
    fileError = open("fuzzMutationEmailError.txt", "w")
    filePass = open("fuzzMutationEmailPass.txt", "w")    

    for i in range(trials):
        mutatedData = mutationFuzzer(seed)       
        
        result = subprocess.run(['python', 'ValidateEmail.py', mutatedData],
                                stdin=subprocess.DEVNULL,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                universal_newlines=True)        
       
        if result.stdout.startswith('Error'):
            runsFail.append((mutatedData + ': ' + result.stdout))
            
            fileError.write((mutatedData + ': ' + result.stdout ))
        else:
            runsPass.append((mutatedData + ': ' + result.stdout ))
            
            filePass.write((mutatedData + ': ' + result.stdout ))
        seed = mutatedData
    fileError.close()
    filePass.close()
    
    

In [372]:
MutateFuzzValidateEmail("my+addresksnjjja@jndfjskd.com", 2)

0
0


In [394]:
for i in range(1):
        
        #coverage = set()
        sys.settrace(traceit)  # Turn on        
        
        result = subprocess.run(['python', 'ValidateEmail.py', 'abhidadadadad@gmadaail.com'],
                                stdin=subprocess.DEVNULL,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                universal_newlines=True)
        sys.settrace(None)  # Turn off
        print(str(len(coverage3)))

1


In [351]:
coverage = set()
sys.settrace(traceit) 
result = subprocess.run(['python', 'ValidateEmail.py', 'jsbajjsab@jmd ks n'],
                                stdin=subprocess.DEVNULL,
                                stdout=subprocess.PIPE,
                                stderr=subprocess.PIPE,
                                universal_newlines=True)
sys.settrace(None) 

In [353]:
str(len(coverage))

'400'

In [291]:
#Mutation fuzzer on validate email
trials = 10
runsPass = []
runsFail = []
fileError = open("fuzzMutationEmailError.txt", "w")
filePass = open("fuzzMutationEmailPass.txt", "w")
initial = "my+address@google.com"

for i in range(trials):
    mutatedData = mutationFuzzer(initial)    
    result = subprocess.run(['python', 'ValidateEmail.py', mutatedData],
                            stdin=subprocess.DEVNULL,
                            stdout=subprocess.PIPE,
                            stderr=subprocess.PIPE,
                            universal_newlines=True) 
    if result.stdout.startswith('Error'):
        runsFail.append((mutatedData + ': ' + result.stdout))
        fileError.write((mutatedData + ': ' + result.stdout))
    else:
        runsPass.append((mutatedData + ': ' + result.stdout))
        filePass.write((mutatedData + ': ' + result.stdout))
    initial = mutatedData
fileError.close()
filePass.close()


In [298]:
import sys


def trace_calls(frame, event, arg):
    if event != 'call':
        return
    co = frame.f_code
    func_name = co.co_name
    if func_name == 'write':
        # Ignore write() calls from printing
        return
    func_line_no = frame.f_lineno
    func_filename = co.co_filename
    if not func_filename.endswith('sys_settrace_call.py'):
        # Ignore calls not in this module
        return
    caller = frame.f_back
    caller_line_no = caller.f_lineno
    caller_filename = caller.f_code.co_filename
    print('* Call to', func_name)
    print('*  on line {} of {}'.format(
        func_line_no, func_filename))
    print('*  from line {} of {}'.format(
        caller_line_no, caller_filename))
    return

In [397]:
#Traceit definition
def traceit(frame, event, arg):
    if event == "line":
        global coverage
        coverage = set()
        #func_filename = frame.f_code.co_filename
        function_name = frame.f_code.co_name
        lineno = frame.f_lineno
        file_name = frame.f_code.co_filename.split('\\')[-1]
        #coverage.append((func_filename, function_name, lineno))
        coverage.add((file_name,function_name, lineno))

    return traceit

In [404]:

sys.settrace(trace_calls)  # Turn on
MutateFuzzValidateEmail("my+address@jsndsdsjd.com",2)
sys.settrace(None)
coverage
len(coverage)

339

In [ ]:
import sys

def execute_and_trace(fun, inp):
    global coverage
    coverage = []
    sys.settrace(traceit)  # Turn on
    fun(inp)
    sys.settrace(None)    # Turn off

inp = "http://www.concordia.ca"
execute_and_trace(is_valid_url, inp)
[cov for cov in coverage]

In [310]:
'c:\\users\\abhishek rajput\\appdata\\local\\programs\\python\\python37\\lib\\site-packages\\IPython\\core\\interactiveshell.py'.split('\\')[-1]

'interactiveshell.py'

In [316]:
coverage = set()
sys.settrace(traceit)  # Turn on
MutateFuzzValidateEmail("my+address@google.com",1)
sys.settrace(None)
[cov for cov in coverage]

[('threading.py', 'wait', 305),
 ('subprocess.py', '_execute_child', 1187),
 ('subprocess.py', '__init__', 699),
 ('contextlib.py', '__init__', 86),
 ('subprocess.py', '_execute_child', 1178),
 ('threading.py', '__init__', 792),
 ('threading.py', 'start', 852),
 ('subprocess.py', '__init__', 765),
 ('subprocess.py', '_cleanup', 227),
 ('subprocess.py', '_execute_child', 1137),
 ('threading.py', '_wait_for_tstate_lock', 1050),
 ('subprocess.py', '_remaining_time', 972),
 ('random.py', 'randrange', 197),
 ('threading.py', 'wait', 297),
 ('<ipython-input-204-1c894bfe4246>', 'mutationFuzzer', 5),
 ('<ipython-input-316-56915bbc3c73>', '<module>', 3),
 ('random.py', '_randbelow', 234),
 ('subprocess.py', '_execute_child', 1170),
 ('threading.py', '__init__', 784),
 ('subprocess.py', '_get_handles', 1042),
 ('threading.py', 'join', 1031),
 ('subprocess.py', '_internal_poll', 1218),
 ('subprocess.py', '__enter__', 820),
 ('subprocess.py', '__init__', 674),
 ('interactiveshell.py', 'run_code', 

In [317]:
len(coverage)

434

In [99]:
''.join('{:08b}'.format(b) for b in 'hi'.encode('utf8'))

'0110100001101001'

In [55]:
''.join(format(ord(x), 'b') for x in 'hi')

'11010001101001'

In [58]:
bits = ''.join(format(ord(x), 'b') for x in 'hi')
''.join('1' if x == '0' else '0' for x in bits)

'00101110010110'

In [106]:
''.join('1' if x == '0' else '0' for x in '0110100001101001')

'1001011110010110'

In [65]:
bits = ''.join('{:08b}'.format(b) for b in 'hi'.encode('utf8'))
flippedBits = ''.join('1' if x == '0' else '0' for x in bits)
n = int(flippedBits, 2)
n.to_bytes((n.bit_length() + 7) // 8, 'big').decode()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x97 in position 0: invalid start byte

In [63]:
''.join(format(ord(x), 'b') for x in 'hi')

'11010001101001'

In [73]:
flippedBits = ''.join('1' if x == '0' else '0' for x in '11010001101001')

In [69]:
n = int(flippedBits, 2) 
n.to_bytes((n.bit_length() + 7) // 8, 'big').decode()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x96 in position 1: invalid start byte

In [113]:
import re
bytes(int(b, 2) for b in re.split('(........)', '0001001011110010110101') if b).decode('utf8')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xf2 in position 1: invalid continuation byte

In [70]:
bits = ''.join('{:08b}'.format(b) for b in 'hi'.encode('utf8'))
print(bits)

0110100001101001


In [66]:
bits = '0110100001101001'
int(bits, 2)


26729

In [18]:

(format(ord('h'), 'b')) & 0xffffffff

TypeError: bad operand type for unary ~: 'str'

In [15]:
'hi'.encode('ascii')

b'hi'

In [1]:
tryyahshasas

NameError: name 'tryyahshasas' is not defined

In [2]:
kdkadkada


NameError: name 'kdkadkada' is not defined